## 1. Libraries

In [ ]:
import pandas as pd
import re
import string
import numpy as np
import torch

from sklearn import preprocessing
from typing import List, Dict

## 2. Load data

In [ ]:
df = pd.read_csv('data.csv')
df.head()

,Title,Link,Author,Category,Content
0,Nie wszystkie ślimaki wyglądają tak samo. Oto ...,https://www.national-geographic.pl/artykul/nie...,Unknown,['Przyroda'],Morza i oceany często są porównywane do przest...
1,"Życie na Ziemi pojawiło się o wiele wcześniej,...",https://www.national-geographic.pl/artykul/zyc...,Unknown,"['Odkrycia', 'Nauka']","Ziemia powstała z mgławicy słonecznej ok. 4,5 ..."
2,Noc spadających gwiazd. Kiedy na niebie widać ...,https://www.national-geographic.pl/artykul/noc...,Unknown,['Kosmos'],Meteoroidy to krążące po kosmosie kawałki skał...
3,„Dziś” w Birmie to jeden wielki znak zapytania...,https://www.national-geographic.pl/artykul/dzi...,"Patronat nad festiwalem „Pięć Smaków""",['Ludzie'],W programie 15. edycji znalazła się m.in. sekc...
4,Co chrumkają świnie? Naukowcy po raz pierwszy ...,https://www.national-geographic.pl/artykul/co-...,Unknown,['Nauka'],"Wszyscy dobrze wiemy, jakie odgłosy wydają świ..."


or

In [ ]:
from google.colab import files
import io

uploaded = files.upload()

Saving data.csv to data.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['data.csv']))
df.head()

,Title,Link,Author,Category,Content
0,Nie wszystkie ślimaki wyglądają tak samo. Oto ...,https://www.national-geographic.pl/artykul/nie...,Unknown,['Przyroda'],Morza i oceany często są porównywane do przest...
1,"Życie na Ziemi pojawiło się o wiele wcześniej,...",https://www.national-geographic.pl/artykul/zyc...,Unknown,"['Odkrycia', 'Nauka']","Ziemia powstała z mgławicy słonecznej ok. 4,5 ..."
2,Noc spadających gwiazd. Kiedy na niebie widać ...,https://www.national-geographic.pl/artykul/noc...,Unknown,['Kosmos'],Meteoroidy to krążące po kosmosie kawałki skał...
3,„Dziś” w Birmie to jeden wielki znak zapytania...,https://www.national-geographic.pl/artykul/dzi...,"Patronat nad festiwalem „Pięć Smaków""",['Ludzie'],W programie 15. edycji znalazła się m.in. sekc...
4,Co chrumkają świnie? Naukowcy po raz pierwszy ...,https://www.national-geographic.pl/artykul/co-...,Unknown,['Nauka'],"Wszyscy dobrze wiemy, jakie odgłosy wydają świ..."


## 3. Clean data

In [ ]:
def clean_basic(text):   
    text = re.sub(r"http\S+", "", text)  
    text = text.lower()
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\xa0', ' ', text)
    text = re.sub(r' / fot. \w* \w*', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r"„", '', text)
    text = re.sub(r"”", '', text)
    text = re.sub(r"\s\s+", ' ', text)
    return text
  
# Removing Emojis
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Removing punctuations
def remove_punctuations(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

def clean_category(text):
  text = re.sub(re.escape('|') + r'National Geographic >', "','", text)
  text = re.sub(r"\s+", '', text)
  return text

def data_cleaning(df):
    df['Content']=df['Content'].apply(lambda x : clean_basic(x))
    df['Content']=df['Content'].apply(lambda x: remove_emoji(x))
    df['Content']=df['Content'].apply(lambda x : remove_punctuations(x))
    df['Category']=df['Category'].apply(lambda x : clean_category(x))
    return df

Example

In [ ]:
text = 'Morza i oceany często są porównywane do przestrzeni kosmicznej. Nie ze względu na ciemności w najgłębszych partiach, a przez to, że niewiele wiemy o większości zamieszkujących je stworzeń. Autorzy badań opublikowanych na łamach czasopisma naukowego „Science Advances” twierdzą, że ponad połowa form życia w oceanach jest niezbadana. Niektóre z gatunków\xa0znanych nauce występują tak rzadko, że ich zaobserwowanie w środowisku naturalnym może wywołać niemałą sensacją. Tak też było w Stanach Zjednoczonych. Ślimaki kojarzą nam się z obślizgłymi stworzonkami, które chowają się w muszli. Rzeczywiście, większość gatunków występujących w Polsce można tak opisać. Całą gromadę tworzy jednak ponad 100 tys. gatunków ślimaków lądowych i morskich. Niektóre z nich mają bardzo nietypowy wygląd. Przekonał się o tym Erick Yanta, Amerykanin wypoczywający razem z żoną na wyspie Mustang w Zatoce Meksykańskiej. \nNiebieski smok nie przypomina ślimaków, jakie spotykamy w Polsce. / fot. James Chasty Para spacerowała po plaży, kiedy zauważyli maleńkie, nie dłuższe niż trzy centymetry, biało-niebieskie stworzonko przylegające do skały. Yanta sfilmował je z bliska. Co to było? Małżeństwo trafiło\xa0na jadowitego ślimaka Glaucus atlanticus, znanego w Polsce jako\xa0niebieski smok lub jaskółka morska. Mimo maleńkich rozmiarów Glaucus atlanticus nie jest bezbronny. W menu ślimaka znajduje się między innymi jadowity żeglarz portugalski. Co ciekawe, zwierzę jest odporne na jego jad. Ba, wręcz przechowuje truciznę swojej ofiary, a następnie wykorzystuje ją\xa0do ochrony przed drapieżnikami i ludźmi. Użądlenie ślimaka jest bardzo bolesne, a w rzadkich przypadkach może zagrażać życiu. Inne objawy poza miejscowym bólem to nudności i wymioty.\xa0 Amerykanin nie ukrywa, że nawet gdyby wcześniej wiedział o tym ryzyku użądlenia przez niebieskiego smoka, i tak zdecydowałby się pooglądać go z bliska. Niebieski smok żywi się też niektórymi meduzami. Sporadycznie ślimaki dopuszczają się kanibalizmu. Cytowany przez CNN Erick Yanta przyznał, że wcześniej widział wiele nietypowych zwierząt morskich. Tym razem miał jednak prawdziwą zagwozdkę. W jej rozwiązaniu pomogli mu internauci. Przypuszczenia amatorów potwierdził David Hicks, profesor na University of Texas w Rio Grande Valley. W rozmowie z CNN dodał, że niebieski smok żyje na powierzchni otwartego oceanu. Ma\xa0niebieski\xa0brzuch\xa0i srebrny\xa0grzbiet, przez co trudno go\xa0zauważyć w wodzie. W ten sposób, dryfując brzuchem do góry, unika\xa0drapieżników. \nNiebieski smok w ataku. / fot. David Maitland'
text

'Morza i oceany często są porównywane do przestrzeni kosmicznej. Nie ze względu na ciemności w najgłębszych partiach, a przez to, że niewiele wiemy o większości zamieszkujących je stworzeń. Autorzy badań opublikowanych na łamach czasopisma naukowego „Science Advances” twierdzą, że ponad połowa form życia w oceanach jest niezbadana. Niektóre z gatunków\xa0znanych nauce występują tak rzadko, że ich zaobserwowanie w środowisku naturalnym może wywołać niemałą sensacją. Tak też było w Stanach Zjednoczonych. Ślimaki kojarzą nam się z obślizgłymi stworzonkami, które chowają się w muszli. Rzeczywiście, większość gatunków występujących w Polsce można tak opisać. Całą gromadę tworzy jednak ponad 100 tys. gatunków ślimaków lądowych i morskich. Niektóre z nich mają bardzo nietypowy wygląd. Przekonał się o tym Erick Yanta, Amerykanin wypoczywający razem z żoną na wyspie Mustang w Zatoce Meksykańskiej. \nNiebieski smok nie przypomina ślimaków, jakie spotykamy w Polsce. / fot. James Chasty Para space

In [ ]:
text = clean_basic(text)
text = remove_emoji(text)
text = remove_punctuations(text)
text

'morza i oceany często są porównywane do przestrzeni kosmicznej nie ze względu na ciemności w najgłębszych partiach a przez to że niewiele wiemy o większości zamieszkujących je stworzeń autorzy badań opublikowanych na łamach czasopisma naukowego science advances twierdzą że ponad połowa form życia w oceanach jest niezbadana niektóre z gatunków znanych nauce występują tak rzadko że ich zaobserwowanie w środowisku naturalnym może wywołać niemałą sensacją tak też było w stanach zjednoczonych ślimaki kojarzą nam się z obślizgłymi stworzonkami które chowają się w muszli rzeczywiście większość gatunków występujących w polsce można tak opisać całą gromadę tworzy jednak ponad tys gatunków ślimaków lądowych i morskich niektóre z nich mają bardzo nietypowy wygląd przekonał się o tym erick yanta amerykanin wypoczywający razem z żoną na wyspie mustang w zatoce meksykańskiej niebieski smok nie przypomina ślimaków jakie spotykamy w polsce para spacerowała po plaży kiedy zauważyli maleńkie nie dłuższ

Data

In [ ]:
df = data_cleaning(df)
df

,Title,Link,Author,Category,Content
0,Nie wszystkie ślimaki wyglądają tak samo. Oto ...,https://www.national-geographic.pl/artykul/nie...,Unknown,['Przyroda'],morza i oceany często są porównywane do przest...
1,"Życie na Ziemi pojawiło się o wiele wcześniej,...",https://www.national-geographic.pl/artykul/zyc...,Unknown,"['Odkrycia','Nauka']",ziemia powstała z mgławicy słonecznej ok mld ...
2,Noc spadających gwiazd. Kiedy na niebie widać ...,https://www.national-geographic.pl/artykul/noc...,Unknown,['Kosmos'],meteoroidy to krążące po kosmosie kawałki skał...
3,„Dziś” w Birmie to jeden wielki znak zapytania...,https://www.national-geographic.pl/artykul/dzi...,"Patronat nad festiwalem „Pięć Smaków""",['Ludzie'],w programie edycji znalazła się min sekcja bi...
4,Co chrumkają świnie? Naukowcy po raz pierwszy ...,https://www.national-geographic.pl/artykul/co-...,Unknown,['Nauka'],wszyscy dobrze wiemy jakie odgłosy wydają świn...
...,...,...,...,...,...
877,Pracujesz nad mięśniami? Nie tylko będziesz si...,https://www.national-geographic.pl/artykul/pra...,Unknown,"['Nauka','Odkrycia']",wiadomości jakie mają dla nas badacze są zaska...
878,"Egzoplanety, czyli planety pozasłoneczne. To o...",https://www.national-geographic.pl/artykul/egz...,Unknown,['Kosmos'],w języku greckim exo oznacza zewnętrzny dlateg...
879,Nasza galaktyka jest prawdopodobnie pełna mart...,https://www.national-geographic.pl/artykul/nas...,Unknown,['Kosmos'],nowe badanie którego autorami są trzej fizycy ...
880,4 zagadki Wszechświata. Czy zapomniana teoria ...,https://www.national-geographic.pl/artykul/skr...,Michał Rolecki,"['Nauka','Kosmos']",ogólna teoria względności opracowana przez alb...


## 4. Prepare dataframe

11 classes:
- 0 - 'Fotografia'
- 1 - 'Historia'
- 2 - 'Kierunki'
- 3 - 'Kosmos'
- 4 - 'Ludzie'
- 5 - 'Nauka'
- 6 - 'Odkrycia'
- 7 - 'Podcasty'
- 8 - 'Promocja'
- 9 - 'Przyroda'
- 10 - 'Smaki Swiata'

### Label encoding

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(['Nauka', 'Ludzie', 'Przyroda', 'Historia', 'Odkrycia', 'Kosmos', 'Promocja', 'Kierunki', 'Podcasty', 'Fotografia', 'Smaki\\u015bwiata'])
list(le.classes_)

['Fotografia',
 'Historia',
 'Kierunki',
 'Kosmos',
 'Ludzie',
 'Nauka',
 'Odkrycia',
 'Podcasty',
 'Promocja',
 'Przyroda',
 'Smaki\\u015bwiata']

Convert string to list of strings

In [ ]:
df['Category'] = df['Category'].apply(eval)

Encode

In [ ]:
df['Numeric'] = df['Category'].apply(le.transform)

In [ ]:
df

,Title,Link,Author,Category,Content,Numeric
0,Nie wszystkie ślimaki wyglądają tak samo. Oto ...,https://www.national-geographic.pl/artykul/nie...,Unknown,[Przyroda],morza i oceany często są porównywane do przest...,[9]
1,"Życie na Ziemi pojawiło się o wiele wcześniej,...",https://www.national-geographic.pl/artykul/zyc...,Unknown,"[Odkrycia, Nauka]",ziemia powstała z mgławicy słonecznej ok mld ...,"[6, 5]"
2,Noc spadających gwiazd. Kiedy na niebie widać ...,https://www.national-geographic.pl/artykul/noc...,Unknown,[Kosmos],meteoroidy to krążące po kosmosie kawałki skał...,[3]
3,„Dziś” w Birmie to jeden wielki znak zapytania...,https://www.national-geographic.pl/artykul/dzi...,"Patronat nad festiwalem „Pięć Smaków""",[Ludzie],w programie edycji znalazła się min sekcja bi...,[4]
4,Co chrumkają świnie? Naukowcy po raz pierwszy ...,https://www.national-geographic.pl/artykul/co-...,Unknown,[Nauka],wszyscy dobrze wiemy jakie odgłosy wydają świn...,[5]
...,...,...,...,...,...,...
877,Pracujesz nad mięśniami? Nie tylko będziesz si...,https://www.national-geographic.pl/artykul/pra...,Unknown,"[Nauka, Odkrycia]",wiadomości jakie mają dla nas badacze są zaska...,"[5, 6]"
878,"Egzoplanety, czyli planety pozasłoneczne. To o...",https://www.national-geographic.pl/artykul/egz...,Unknown,[Kosmos],w języku greckim exo oznacza zewnętrzny dlateg...,[3]
879,Nasza galaktyka jest prawdopodobnie pełna mart...,https://www.national-geographic.pl/artykul/nas...,Unknown,[Kosmos],nowe badanie którego autorami są trzej fizycy ...,[3]
880,4 zagadki Wszechświata. Czy zapomniana teoria ...,https://www.national-geographic.pl/artykul/skr...,Michał Rolecki,"[Nauka, Kosmos]",ogólna teoria względności opracowana przez alb...,"[5, 3]"


### One-hot encoding

In [ ]:
def to_multilabel_vector(instances: List[dict], vector_size: int = 11):
  vector = np.zeros(vector_size)
  for i in instances:
    vector[i] = 1
  return vector

In [ ]:
df['onehot'] = df['Numeric'].apply(to_multilabel_vector)
df['onehot']

0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1      [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...
2      [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3      [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
4      [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
                             ...                        
877    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...
878    [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
879    [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
880    [0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
881    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: onehot, Length: 882, dtype: object

### Final processing

In [ ]:
onehot_array = np.stack(df.onehot.values)
categories = pd.DataFrame({'Fotografia': onehot_array[:, 0], 'Historia': onehot_array[:, 1], 'Kierunki': onehot_array[:, 2],
              'Kosmos': onehot_array[:, 3], 'Ludzie': onehot_array[:, 4], 'Nauka': onehot_array[:, 5], 'Odkrycia': onehot_array[:, 6],
              'Podcasty': onehot_array[:, 7], 'Promocja': onehot_array[:, 8], 'Przyroda': onehot_array[:, 9], 'Smaki': onehot_array[:, 10]})

In [ ]:
categories.sum(axis=0)

Fotografia      1.0
Historia      199.0
Kierunki        1.0
Kosmos        200.0
Ludzie        207.0
Nauka         212.0
Odkrycia      235.0
Podcasty        2.0
Promocja        5.0
Przyroda      228.0
Smaki           1.0
dtype: float64

In [ ]:
categories = categories.drop(columns=['Fotografia', 'Kierunki', 'Podcasty', 'Promocja', 'Smaki'])

In [ ]:
df_final = pd.concat([df.Content, categories], axis=1)
df_final

,Content,Historia,Kosmos,Ludzie,Nauka,Odkrycia,Przyroda
0,morza i oceany często są porównywane do przest...,0.0,0.0,0.0,0.0,0.0,1.0
1,ziemia powstała z mgławicy słonecznej ok mld ...,0.0,0.0,0.0,1.0,1.0,0.0
2,meteoroidy to krążące po kosmosie kawałki skał...,0.0,1.0,0.0,0.0,0.0,0.0
3,w programie edycji znalazła się min sekcja bi...,0.0,0.0,1.0,0.0,0.0,0.0
4,wszyscy dobrze wiemy jakie odgłosy wydają świn...,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
877,wiadomości jakie mają dla nas badacze są zaska...,0.0,0.0,0.0,1.0,1.0,0.0
878,w języku greckim exo oznacza zewnętrzny dlateg...,0.0,1.0,0.0,0.0,0.0,0.0
879,nowe badanie którego autorami są trzej fizycy ...,0.0,1.0,0.0,0.0,0.0,0.0
880,ogólna teoria względności opracowana przez alb...,0.0,1.0,0.0,1.0,0.0,0.0


In [ ]:
df_final.to_csv(r'data_final.csv', index=False)